In [ ]:
import numpy as np
import random

In [ ]:
# Define the alphabet
alphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 .'

codewords = []
for i in range(32):
    p = np.zeros(64)
    p[2*i] = 32
    codewords.append(p)
    q = np.zeros(64)
    q[2*i] = -32
    codewords.append(q)

CODEBOOK = {}
for i in range(64):
    CODEBOOK[alphabet[i]] = codewords[i]
for i in range(32):
    p = np.zeros(64)
    p[2*i+1] = 200
    CODEBOOK['xp'+str(i)] = p
    q = np.zeros(64)
    q[2*i+1] = -200
    CODEBOOK['xq'+str(i)] = q


In [ ]:
# Simulate the channel
def channel(x):
    n = x.size
    B = random.choice([0, 1])
    sigma_sq = 25
    Z = np.random.normal(0, np.sqrt(sigma_sq), 2 * n)
    X = np.zeros(2 * n)
    if B == 1:
        X[:n] = x
    else:
        X[n:] = x
    Y = X + Z
    Y = np.reshape(Y, (-1))
    return Y

# Transmitter
message = "111aaaaaaaaaaaa.aaaa123aaaaaaaaaaaaaaaba"
#message = alphabet[-40:]
print("Original Message:", message)

transmitted_signal = np.zeros(40*64)
for i in range(40):
    transmitted_signal[i*64:(i+1)*64] = CODEBOOK[message[i]]

np.savetxt('input_signal.txt', transmitted_signal)

#received_signal = channel(transmitted_signal)
    #np.savetxt('received_signal.txt', received_signal)
#print("Received Signal:", received_signal)

In [ ]:
# Receiver
    #received_signal = np.loadtxt('received_signal.txt')
received_signal = []
with open('output.txt') as f:
    for line in f:
        received_signal.append(float(line))
message_code1 = []
message_code2 = []
for i in range(40):
    message_code1.append(received_signal[i*64:(i+1)*64])
    message_code2.append(received_signal[len(received_signal)//2+i*64:len(received_signal)//2+(i+1)*64])
message_bin = []
for i in range(40):
    char_bin = ''
    y1 = message_code1[i]
    y2 = message_code2[i]
    i1 = np.argmin([np.linalg.norm(y1 - c) for c in CODEBOOK.values()])
    char1 = alphabet[i1]
    i2 = np.argmin([np.linalg.norm(y2 - c) for c in CODEBOOK.values()])
    char2 = alphabet[i2]
    d1 = np.linalg.norm(y1 - CODEBOOK[char1]) ** 2 / 25 + np.linalg.norm(y2) ** 2 / 25
    d2 = np.linalg.norm(y2 - CODEBOOK[char2]) ** 2 / 25 + np.linalg.norm(y1) ** 2 / 25
    if d1 < d2:
        char_bin = char1
    else:
        char_bin = char2  
    message_bin.append(char_bin)

msg = ''
for i in range(40):
    msg += (message_bin[i])

print(msg)
